# Створення додатку з Retrieval Augmented Generation (RAG)

Одним з найпотужніших застосувань, які забезпечують LLM, є складні чат-боти для запитань і відповідей (Q&A). Це програми, які можуть відповідати на запитання про конкретну інформацію з джерела користувача. Ці програми використовують техніку, відому як Retrieval Augmented Generation, або RAG.

В цьому розділі ми подивимось як створити просту програму для Q&A на основі текстового джерела даних. Під час цього ми розглянемо типову архітектуру Q&A та в кінці я надам додаткові ресурси для більш просунутих технік Q&A. Ми також побачимо, як LangSmith може допомогти нам відстежувати та розуміти нашу програму. LangSmith стане все більш корисним, оскільки наша програма зростатиме в складності.

## Що таке RAG?

RAG — це техніка, що дозволяє доповнювати знання LLM додатковими даними.

LLM можуть міркувати про широкий спектр тем, але їхні знання обмежені публічними даними до певного моменту часу, на яких вони були навчені (cutoff). Якщо ви хочете створити AI програми, які можуть враховувати приватні дані або дані, введені після дати відсічення моделі, вам потрібно доповнити знання моделі конкретною інформацією, яка їй потрібна. Процес внесення та вставки відповідної інформації в запит моделі відомий як Retrieval Augmented Generation (RAG).

LangChain має ряд компонентів, призначених для допомоги у створенні програм Q&A, а також програм з використанням RAG.

**Примітка**: Тут ми зосереджуємося на Q&A для неструктурованих даних. Якщо вас цікавить RAG для структурованих даних - можна ознайомитись з прикладом тут [питання і відповіді на SQL дані](https://python.langchain.com/docs/tutorials/sql_qa/).

## Концепції
Типова програма з використанням RAG має дві основні компоненти:

**Індексація**: конвеєр для завантаження даних з джерела та їх індексації. *Це зазвичай відбувається офлайн. Тобто не прям коли ми викликаємо модель для відповідей на наші питання*

**Пошук і генерація**: фактичний ланцюг RAG, який бере запит користувача під час виконання та отримує відповідні дані з індексу, а потім передає їх моделі.

Найбільш поширена повна послідовність від сирих даних до відповіді виглядає так:

### Індексація
1. **Завантажити**: Спочатку потрібно завантажити наші дані. Це робиться за допомогою [Document Loaders](https://python.langchain.com/docs/concepts/document_loaders/).
2. **Розділити**: [Text splitters](https://python.langchain.com/docs/concepts/text_splitters/) розбивають великі `Documents` на менші частини. Це корисно як для індексації даних, так і для передачі їх у модель, оскільки великі частини важче шукати і вони не помістяться в обмежене вікно контексту моделі.
3. **Зберегти**: Нам потрібно місце для зберігання та індексації наших частин, щоб їх можна було шукати пізніше. Це часто робиться за допомогою [VectorStore](https://python.langchain.com/docs/concepts/vectorstores/) та [Embeddings](https://python.langchain.com/docs/concepts/embedding_models/) моделі.

![index_diagram](https://python.langchain.com/assets/images/rag_indexing-8160f90a90a33253d0154659cf7d453f.png)

### Пошук і генерація
4. **Отримання**: З урахуванням введення користувача, відповідні частини отримуються зі сховища за допомогою [Retriever](https://python.langchain.com/docs/concepts/retrievers/).
5. **Генерація**: [ChatModel](https://python.langchain.com/docs/concepts/chat_models/) / [LLM](https://python.langchain.com/docs/concepts/text_llms/) генерує відповідь, використовуючи запит, що включає як запит, так і отримані дані.

![retrieval_diagram](https://python.langchain.com/assets/images/rag_retrieval_generation-1046a4668d6bb08786ef73c56d4f228a.png)



Встановлюємо залежності:

In [ ]:
!pip install --quiet --upgrade langchain langchain-community langchain-chroma langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.7/615.7 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 35.7 MB/s eta 0:00:00
 

### LangSmith

Багато з додатків, які ви створюєте з LangChain, міститимуть кілька етапів з кількома викликами LLM (Large Language Model).
Оскільки ці додатки стають все більш складними, стає важливим мати можливість перевірити, що саме відбувається всередині вашого ланцюга або агента.
Найкращий спосіб зробити це - за допомогою [LangSmith](https://smith.langchain.com).

LangSmith - це універсальна платформа для розробників для кожного етапу життєвого циклу додатків на основі LLM. LangSmith допомагає вам відстежувати та оцінювати ваші LLM-apps та інтелектуальних агентів, щоб допомогти вам перейти від прототипу до production.

Після реєстрації за вказаним вище посиланням, переконайтеся, що ви налаштували свої змінні середовища для початку трейсингу:



In [ ]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [ ]:
import json
import os

with open('creds.json') as file:
  creds = json.load(file)

os.environ["OPENAI_API_KEY"] = creds["OPENAI_API_KEY"]
os.environ["LANGCHAIN_API_KEY"] = creds["LANGCHAIN_API_KEY"]

## Попередній перегляд

У цьому уроці ми створимо додаток, який відповідає на запитання про вміст веб-сайту. Конкретний веб-сайт, який ми будемо використовувати, - це [LLM Powered Autonomous
Agents](https://lilianweng.github.io/posts/2023-06-23-agent/) блог-пост
Ліліан Венг.

Ми можемо створити простий індексуючий конвеєр і RAG (Retrieval-Augmented Generation) ланцюг  за 20
рядків коду:

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4")

In [ ]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Завантажуємо, розбиваємо на частини та індексуємо вміст блогу.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [ ]:
# Отримуємо та генеруємо, використовуючи відповідні фрагменти блогу.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition is a process where complex tasks are broken down into smaller, simpler steps or subtasks. Techniques like Chain of Thought (CoT) and Tree of Thoughts are used to decompose tasks, guiding a model to think step by step, explore multiple reasoning possibilities, and thus transform big tasks into multiple manageable tasks. This process can be facilitated through different methods such as simple prompting, task-specific instructions, or human inputs.'

In [ ]:
# наводимо лад
vectorstore.delete_collection()

Перегляньте [LangSmith trace](https://smith.langchain.com/public/1c6ca97e-445b-4d00-84b4-c7befcbc59fe/r).

## Детальний огляд

Давайте пройдемося по наведеному коду крок за кроком, щоб дійсно зрозуміти, що відбувається.

## 1. Індексація: Завантаження

Спочатку нам потрібно завантажити вміст блогу. Для цього ми можемо використовувати
DocumentLoaders,
які є об'єктами, що завантажують дані з джерела і повертають список
[Documents](https://python.langchain.com/api_reference/core/documents/langchain_core.documents.base.Document.html).
`Document` - це об'єкт з деяким `page_content` (str) та `metadata`
(dict).

У цьому випадку ми використаємо
[WebBaseLoader](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.web_base.WebBaseLoader.html),
який використовує `urllib` для завантаження HTML з веб-URL і `BeautifulSoup` для
перетворення його в текст. Ми можемо налаштувати перетворення HTML -> текст, передавши
параметри в парсер `BeautifulSoup` через `bs_kwargs` (див.
[документацію BeautifulSoup](https://beautiful-soup-4.readthedocs.io/en/latest/#beautifulsoup)).

У цьому випадку лише HTML-теги з класом “post-content”, “post-title” або
“post-header” є релевантними, тому ми видалимо всі інші.

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Лишаємо лише назву, заголовки та вміст допису з повного HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

len(docs[0].page_content)

43131

In [ ]:
print(docs[0].page_content[:500])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


In [ ]:
type(docs)

list

### Заглиблення

`DocumentLoader`: Об'єкт, який завантажує дані з джерела у вигляді списку `Documents`.

- [Документація](https://python.langchain.com/docs/how_to/#document-loaders):
  Докладна документація про те, як використовувати `DocumentLoaders`.
- [Інтеграції](https://python.langchain.com/docs/integrations/document_loaders/): 160+
  інтеграцій на вибір.
- [Інтерфейс](https://python.langchain.com/api_reference/core/document_loaders/langchain_core.document_loaders.base.BaseLoader.html):
  API-довідка для базового інтерфейсу.


## 2. Індексація: Розділення


Наш завантажений документ містить понад 42 тисячі символів, що занадто багато, щоб вміститися у вікно контексту багатьох моделей. Навіть для тих моделей, які можуть вмістити весь пост у своєму вікні контексту, моделі можуть мати труднощі з пошуком інформації у дуже довгих введеннях.

Перевірити величину контекстного вікна в OpenAI моделей можна тут: https://platform.openai.com/docs/models

Щоб впоратися з цим, ми розділимо `Document` на частини для вбудовування (embedding) та зберігання векторів. Це має допомогти нам отримувати лише найрелевантніші частини блогу під час обробки запиту від користувача.

У цьому випадку ми розділимо наші документи на частини по 1000 символів з 200 символами перекриття між частинами. Перекриття допомагає зменшити ймовірність відокремлення твердження від важливого контексту, пов'язаного з ним. Ми використовуємо
[RecursiveCharacterTextSplitter](https://python.langchain.com/docs/how_to/recursive_text_splitter/),
який рекурсивно розділяє документ, використовуючи загальні роздільники, такі як нові рядки, поки кожна частина не досягне відповідного розміру. Це рекомендований роздільник тексту для загальних випадків використання тексту.

Ми встановлюємо `add_start_index=True`, щоб індекс символів, з якого починається кожен розділений документ у початковому документі, зберігався як атрибут метаданих "start_index".

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

66

In [ ]:
len(all_splits[0].page_content)

969

In [ ]:
all_splits[0]

Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 8}, page_content='LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from 

In [ ]:
all_splits[10].metadata

{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/',
 'start_index': 7056}

### Заглиблення

`TextSplitter`: Об'єкт, який розділяє список `Document` на менші частини. Підклас `DocumentTransformer`.

- Дізнатись більше про різні методи розділення тексту ви можете прочитавши [документацію](https://python.langchain.com/docs/how_to/#text-splitters)
- [Код (py або js)](https://python.langchain.com/docs/integrations/document_loaders/source_code/)
- [Наукові статті](https://python.langchain.com/docs/integrations/document_loaders/grobid/)
- [Інтерфейс](https://python.langchain.com/api_reference/text_splitters/base/langchain_text_splitters.base.TextSplitter.html): API-довідка для базового інтерфейсу.

`DocumentTransformer`: Об'єкт, який виконує трансформацію над списком об'єктів `Document`.

- [Документація](https://python.langchain.com/docs/how_to/#text-splitters): Докладна документація про те, як використовувати `DocumentTransformers`
- [Інтеграції](https://python.langchain.com/docs/integrations/document_transformers/)
- [Інтерфейс](https://python.langchain.com/api_reference/core/documents/langchain_core.documents.transformers.BaseDocumentTransformer.html): API-довідка для базового інтерфейсу.

## 3. Індексація: Зберігання

Тепер нам потрібно проіндексувати наші 66 текстових частин, щоб ми могли шукати їх під час виконання.

Найпоширеніший спосіб зробити це — вбудувати вміст кожного розділеного документа і вставити ці ембедінги в векторну базу даних (або векторне сховище). Коли ми хочемо шукати серед наших розділів, ми беремо текстовий запит на пошук, вбудовуємо його і виконуємо певний вид пошуку на "схожість", щоб визначити збережені розділи з найбільш схожими ембедінгами на наш ембедінг запиту. Найпростіша міра схожості — косинусна схожість — ми вимірюємо косинус кута між кожною парою ембедінгів (які є векторами високої розмірності).

![](https://miro.medium.com/v2/resize:fit:1400/1*-AL-kK8HzK5lw84xr1cSvw.png)

Ми можемо вбудувати і зберегти всі наші розділи документів одною командою, використовуючи векторне сховище [Chroma](/https://python.langchain.com/docs/integrations/vectorstores/chroma/) та модель [OpenAIEmbeddings](https://python.langchain.com/docs/integrations/text_embedding/openai/).

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

In [ ]:
vectorstore

### Заглиблення

`Embeddings`: Обгортка навколо моделі векторизації тексту, що використовується для перетворення тексту на вектори.

- [Документація](https://python.langchain.com/docs/how_to/embed_text/): Докладна документація про те, як використовувати вектори.
- [Інтеграції](https://python.langchain.com/docs/integrations/text_embedding/): Понад 30 інтеграцій на вибір.
- [Інтерфейс](https://python.langchain.com/api_reference/core/embeddings/langchain_core.embeddings.Embeddings.html): API-довідка для базового інтерфейсу.

`VectorStore`: Обгортка навколо векторної бази даних, що використовується для зберігання та запитів до векторів. Векторн база зберігає вектори ефективніше за звичайну.

- [Документація](https://python.langchain.com/docs/how_to/vectorstores/): Докладна документація про те, як використовувати векторні сховища.
- [Інтеграції](https://python.langchain.com/docs/integrations/vectorstores/): Понад 40 інтеграцій на вибір.
- [Інтерфейс](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.VectorStore.html): API-довідка для базового інтерфейсу.

Це завершує частину **Індексації** в конвеєрі. На цьому етапі у нас є векторне сховище, що містить розділені частини нашого блогу, до якого можна робити запити. Відповідаючи на запит користувача, ми повинні мати можливість повернути фрагменти блогу, які відповідають на запит.

## 4. Retrieval & Generation: Retrieve

Тепер давайте напишемо фактичну логіку програми. Ми хочемо створити просту програму, яка приймає запит користувача, шукає документи, що стосуються цього запиту, передає знайдені документи та початкове питання моделі і повертає відповідь.

Спочатку нам потрібно визначити нашу логіку для пошуку документів. LangChain визначає
[Retriever](https://python.langchain.com/docs/concepts/#retrievers/) інтерфейс,
який обгортає індекс, що може повертати відповідні `Documents` за заданим рядковим запитом.

Найпоширенішим типом `Retriever` є
[VectorStoreRetriever](https://python.langchain.com/docs/how_to/vectorstore_retriever/),
який використовує можливості пошуку подібності векторного сховища для полегшення витягування потрібних документів. Будь-який `VectorStore` можна легко перетворити на `Retriever` за допомогою `VectorStore.as_retriever()`:

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("What are the approaches to Task Decomposition?")

len(retrieved_docs)

6

In [ ]:
print(retrieved_docs[0].page_content)

Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.
Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.


In [ ]:
print(retrieved_docs[-1].page_content)

Another quite distinct approach, LLM+P (Liu et al. 2023), involves relying on an external classical planner to do long-horizon planning. This approach utilizes the Planning Domain Definition Language (PDDL) as an intermediate interface to describe the planning problem. In this process, LLM (1) translates the problem into “Problem PDDL”, then (2) requests a classical planner to generate a PDDL plan based on an existing “Domain PDDL”, and finally (3) translates the PDDL plan back into natural language. Essentially, the planning step is outsourced to an external tool, assuming the availability of domain-specific PDDL and a suitable planner which is common in certain robotic setups but not in many other domains.
Self-Reflection#
Self-reflection is a vital aspect that allows autonomous agents to improve iteratively by refining past action decisions and correcting previous mistakes. It plays a crucial role in real-world tasks where trial and error are inevitable.


### Заглиблення

Векторні сховища зазвичай використовуються для отримання інформації, але існують й інші способи виконання запитів.

`Retriever`: Об'єкт, який повертає `Document` (документи) за текстовим запитом.

- [Документація](https://python.langchain.com/docs/how_to/#retrievers): Додаткова документація про інтерфейс та вбудовані техніки отримання інформації. Деякі з них включають:
  - `MultiQueryRetriever` [генерує варіанти вхідного запитання](https://python.langchain.com/docs/how_to/MultiQueryRetriever) для покращення retrieval hit rate (кількості влучно отриманих чанків документів).
  - `MultiVectorRetriever` натомість генерує [варіанти векторів](https://python.langchain.com/docs/how_to/multi_vector), також з метою покращення retrieval hit rate..
  - `Maximal marginal relevance` максимізує [релевантність та різноманітність](https://www.cs.cmu.edu/~jgc/publication/The_Use_MMR_Diversity_Based_LTMIR_1998.pdf) серед отриманих документів, щоб уникнути повторення контексту.
  - Документи можуть бути відфільтровані під час отримання інформації з векторного сховища за допомогою фільтрів метаданих, таких як у [Self Query Retriever](https://python.langchain.com/docs/how_to/self_query).
- [Інтеграції](https://python.langchain.com/docs/integrations/retrievers/): Інтеграції з сервісами отримання інформації.
- [Інтерфейс](https://python.langchain.com/api_reference/core/retrievers/langchain_core.retrievers.BaseRetriever.html): API-довідка для базового інтерфейсу.

## 5. Retrieval & Generation: Generate

Давайте об'єднаємо все в ланцюг, який приймає запитання, отримує відповідні документи, формує запит, передає його в модель і аналізує вихідні дані.

Ми використаємо модель чату gpt-4o-mini від OpenAI, але могли б використати будь-яку `LLM` або `ChatModel` з тих, що дає можливість використовувати LangChain.

Ми використаємо запит для RAG, який зберігається в хабі запитів LangChain ([тут](https://smith.langchain.com/hub/rlm/rag-prompt)).

In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [ ]:
print(prompt.messages[0].prompt.template)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:


In [ ]:
example_messages = prompt.invoke(
    {"context": "my interesting context", "question": "my unique question"}
).to_messages()

example_messages

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: my unique question \nContext: my interesting context \nAnswer:", additional_kwargs={}, response_metadata={})]

In [ ]:
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: my unique question 
Context: my interesting context 
Answer:


Ми використаємо протокол [LCEL Runnable](https://python.langchain.com/docs/concepts#langchain-expression-language-lcel) для визначення ланцюга, що дозволить нам

- з'єднувати компоненти та функції прозорим способом
- автоматично відстежувати наш ланцюг у LangSmith
- отримувати потокове, асинхронне та пакетне викликання з коробки.

Ось реалізація:

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("What is Task Decomposition?"):
    print(chunk, end="", flush=True)

Task Decomposition is the process of breaking down complex tasks into smaller, simpler steps. This can be achieved through techniques like Chain of Thought and Tree of Thoughts, which transform large assignments into multiple manageable tasks. Decomposition can be guided by simple prompting, task-specific instructions, or human inputs.

Давайте розглянемо LCEL, щоб зрозуміти, що відбувається.

По-перше: кожен з цих компонентів (`retriever`, `prompt`, `llm` тощо) є екземплярами [Runnable](https://python.langchain.com/docs/concepts#langchain-expression-language-lcel). Це означає, що вони реалізують однакові методи - такі як синхронний і асинхронний `.invoke`, `.stream` або `.batch` - що полегшує їх з'єднання. Їх можна з'єднувати в [RunnableSequence](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.RunnableSequence.html) - ще один Runnable - за допомогою оператора `|`.

LangChain автоматично перетворює певні об'єкти на runnable, коли стикається з оператором `|`. Тут `format_docs` перетворюється на [RunnableLambda](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.RunnableLambda.html), а словник з `"context"` і `"question"` перетворюється на [RunnableParallel](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.RunnableParallel.html). Деталі менш важливі, ніж загальна ідея, яка полягає в тому, що кожен об'єкт у ланцюзі є Runnable.

Давайте прослідкуємо, як вхідне питання проходить через вищезгадані runnable.

Як ми бачили вище, вхід для `prompt` очікується у вигляді словника з ключами `"context"` і `"question"`. Отже, перший елемент цього ланцюга створює runnable, які обчислять обидва ці значення з вхідного питання:
- `retriever | format_docs` передає питання через retriever, генеруючи [Document](https://python.langchain.com/api_reference/core/documents/langchain_core.documents.base.Document.html) об'єкти, а потім до `format_docs`, щоб згенерувати рядки;
- `RunnablePassthrough()` передає вхідне питання без змін.

Тобто, якщо ви створите
```python
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
)
```
Тоді `chain.invoke(question)` створить відформатований запит, готовий для інференції. (Примітка: під час розробки з LCEL може бути практично тестувати з підланцюгами, як цей.)

Останні кроки ланцюга - це `llm`, який виконує інференцію, і `StrOutputParser()`, який просто витягує рядковий вміст з виходу LLM.

Ви можете проаналізувати окремі кроки цього ланцюга через його [LangSmith trace](https://smith.langchain.com/public/1799e8db-8a6d-4eb2-84d5-46e8d7d5a99b/r).

### Вбудовані ланцюги (з базового функціоналу LangChain)

Якщо потрібно, LangChain включає зручні функції, які реалізують вищезгаданий LCEL. Ми складаємо дві функції:

- [create_stuff_documents_chain](https://python.langchain.com/api_reference/langchain/chains/langchain.chains.combine_documents.stuff.create_stuff_documents_chain.html) визначає, як отриманий контекст подається в запит і LLM. У цьому випадку ми "заповнимо" вміст у запит - тобто, ми включимо весь отриманий контекст без будь-якого узагальнення або іншої обробки. Це в основному реалізує наш вище `rag_chain`, з вхідними ключами `context` і `input` - він генерує відповідь, використовуючи отриманий контекст і запит.
- [create_retrieval_chain](https://python.langchain.com/api_reference/langchain/chains/langchain.chains.retrieval.create_retrieval_chain.html) додає крок отримання (retrieval) і передає отриманий контекст через ланцюг, надаючи його разом з фінальною відповіддю. Він має вхідний ключ `input` і включає `input`, `context` і `answer` у своєму виході.

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "What is Task Decomposition?"})
print(response["answer"])

Task Decomposition is the process of breaking down a complex task into smaller, simpler steps. Techniques such as Chain of Thought (CoT) and Tree of Thoughts are used to enhance model performance on complex tasks. They instruct the model to think step by step, and decompose hard tasks into manageable ones, thus providing an interpretation of the model’s thinking process.


#### Одержання джерел
Часто в Q&A додатках важливо показувати користувачам джерела, які були використані для генерації відповіді. Вбудована функція LangChain `create_retrieval_chain` передасть отримані джерела документів у вихідні дані під ключем `"context"`:

In [ ]:
for document in response["context"]:
    print(document)
    print()

page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.
Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposition#
Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.' metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 1585}

page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS 

### Заглиблення

#### Вибір моделі

`ChatModel`: Модель чату на основі LLM. Приймає послідовність повідомлень і повертає повідомлення.

- [Документація](https://python.langchain.com/docs/how_to#chat-models)
- [Інтеграції](https://python.langchain.com/docs/integrations/chat/): більше 25 інтеграцій на вибір.
- [Інтерфейс](https://python.langchain.com/api_reference/core/language_models/langchain_core.language_models.chat_models.BaseChatModel.html): API-довідка для базового інтерфейсу.

`LLM`: Модель LLM з текстом на вході та текстом на виході. Приймає рядок і повертає рядок.

- [Документація](https://python.langchain.com/docs/how_to#llms)
- [Інтеграції](https://python.langchain.com/docs/integrations/llms): більше 75 інтеграцій на вибір.
- [Інтерфейс](https://python.langchain.com/api_reference/core/language_models/langchain_core.language_models.llms.BaseLLM.html): API-довідка для базового інтерфейсу.

Дивіться посібник з RAG з локально запущеними моделями [тут](https://python.langchain.com/docs/tutorials/local_rag).

#### Налаштування запиту

Як показано вище, ми можемо завантажувати запити (наприклад, [цей запит RAG](https://smith.langchain.com/hub/rlm/rag-prompt)) з хабу запитів. Запит також можна легко налаштувати:

In [ ]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always ask next question at the end to clarify what else user might be interested in.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

'Task decomposition is a process in which a complex task is broken down into smaller, simpler steps or subtasks. Techniques like Chain of Thought (CoT) and Tree of Thoughts help in this process by systematically decomposing tasks and exploring multiple reasoning possibilities. This allows an autonomous agent to manage and execute larger tasks more efficiently. What other aspects of the LLM-powered autonomous agent system would you like to know about?'

Перегляньте [LangSmith trace](https://smith.langchain.com/public/da23c4d8-3b33-47fd-84df-a3a582eedf84/r)

## Наступні кроки

Ми розглянули етапи створення базового додатку для запитань і відповідей (Q&A) на основі даних:

- Завантаження даних за допомогою [Document Loader](https://python.langchain.com/docs/concepts#document-loaders)
- Розбиття індексованих даних за допомогою [Text Splitter](https://python.langchain.com/docs/concepts#text-splitters) для зручнішого використання моделлю
- [Вбудовування даних](https://python.langchain.com/docs/concepts#embedding-models) та зберігання даних у [vectorstore](https://python.langchain.com/docs/how_to/vectorstores)
- [Отримання](https://python.langchain.com/docs/concepts#retrievers) раніше збережених частин у відповідь на вхідні запитання
- Генерація відповіді, використовуючи отримані частини як контекст

Є безліч функцій, інтеграцій та розширень, які можна дослідити в кожному з вищезазначених розділів. Разом з **Go deeper** джерелами, згаданими вище, хорошими наступними кроками є:

- [Повернення джерел](https://python.langchain.com/docs/how_to/qa_sources): Дізнайтеся, як повертати джерела документів
- [Стрімінг](https://python.langchain.com/docs/how_to/streaming): Дізнайтеся, як стрімити виходи та проміжні етапи
- [Додати історію чату](https://python.langchain.com/docs/how_to/message_history): Дізнайтеся, як додати історію чату до вашого додатку
- [Концептуальний посібник з отримання](https://python.langchain.com/docs/concepts/retrieval): Загальний огляд специфічних технік отримання (retrieval)
- [Створити локальний RAG додаток](https://python.langchain.com/docs/tutorials/local_rag): Створіть додаток, подібний до наведеного вище, використовуючи всі локальні компоненти